In [1]:
# Code is working as of Jun 22

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from multiprocessing import Pool
import sys
sys.path.append('../')

In [3]:
from funcs.analysis.obj_survey import obj_survey
from funcs.preprocessing.parse import intersection, filter_data
from funcs.analysis.plotting import plot_magerr_hist, plot_mag_dist

# Merging lcs from unaveraged data

In [4]:
obj = 'calibStars'
ID  = 'uid_s'
# obj = 'qsos'
# ID  = 'uid'

In [5]:
nrows = None
bounds={'mag_ps':(15,25),'magerr':(0,2)}

In [6]:
colors = pd.read_csv('../../data/computed/{}/colors_sdss.csv'.format(obj), index_col=0)

In [7]:
sdss = obj_survey('sdss', obj, ID)
sdss.read_in_raw(nrows, melted=True, save=False)
sdss.transform_to_ps(colors=colors, color='g-r', system='tonry')
sdss.df = filter_data(sdss.df, bounds)#, bands = 'ugriz') # apply filter after transformation so any bad transformations are removed
# sdss.residual_raw({'g':0.0148, 'r':0.0049, 'i':0.0198, 'z':0.042}) # apply residuals

/disk1/hrb/anaconda3/envs/astroconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


num obs before: 43,204,592
num obs after:  43,125,866


In [8]:
# def apply_max_z(group):
#     mag = group['mag_ps'].values
#     z = (mag-mag.mean())/mag.std()
#     max_z = max(abs(z))
#     return max_z

In [9]:
ps = obj_survey('ps', obj, ID)
ps.read_in_raw(nrows)
ps.df['mag_ps'] = ps.df['mag']
ps.df = filter_data(ps.df, bounds)

/disk1/hrb/anaconda3/envs/astroconda/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


num obs before: 41,693,053
num obs after:  41,676,220


In [10]:
ztf = obj_survey('ztf', obj, ID)
# ztf.pivot()
ztf.read_in_raw(nrows)
ztf.transform_to_ps(colors=colors)
ztf.df = filter_data(ztf.df, bounds) # apply filter after transformation so any bad transformations are removed
# ztf.residual_raw({'g':0.0074, 'r':-0.0099, 'i':0}) # apply residuals

attempting parallel reading


/disk1/hrb/anaconda3/envs/astroconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/disk1/hrb/anaconda3/envs/astroconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/disk1/hrb/anaconda3/envs/astroconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/disk1/hrb/anaconda3/envs/astroconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


completed parallel reading
num obs before: 62,056,352
num obs after:  62,056,295


# group together observations with ∆t < 1 day?

## Merge and save lightcurves

In [11]:
sdss.df['catalogue'] = 5
ps  .df['catalogue'] = 7
ztf .df['catalogue'] = 11
cols = ['mjd','mag','mag_ps','magerr','catalogue']
master = pd.concat([sdss.df[cols].reset_index(), ps.df[cols].reset_index(), ztf.df[cols].reset_index()], axis = 0, ignore_index = True).astype({sdss.ID:'int'}).set_index(ID)

In [12]:
master

,filtercode,mjd,mag,mag_ps,magerr,catalogue
uid_s,,,,,,
1,g,51465.129240,22.053910,21.859878,0.113186,5
1,r,51465.129240,20.783370,20.773518,0.054754,5
1,i,51465.129240,20.162650,20.150477,0.047815,5
1,z,51465.129240,20.158980,20.195187,0.181666,5
1,g,52053.425802,22.370310,22.145209,0.149446,5
...,...,...,...,...,...,...
1006830,r,58347.453947,21.370272,21.521317,0.229004,11
1006830,r,58345.497986,20.887083,21.025773,0.191271,11
1006830,r,58345.483426,21.027519,21.161656,0.202238,11


In [13]:
# Remove objects with a single observation
# faster to do .duplicated() ?
value_counts = master.index.value_counts()
uids = value_counts[value_counts==1].index
master = master[~master.index.isin(uids)]

In [14]:
# Drop nan mag entries
master = master[~master['mag_ps'].isna()]

In [15]:
master = master.sort_values([ID,'mjd'])

In [16]:
# If we are getting 'cannot allocate memory' then do one at a time.
# master = master[master['filtercode']=='r']

In [19]:
# If we want split up bands
def save(args):
    i, chunk = args
    chunk.to_csv('/disk1/hrb/python/data/merged/{}/{}_band/unclean/lc_{}.csv'.format(obj, band, i))

for band in 'giz':
    chunks = np.array_split(master[master['filtercode']==band].drop(columns='filtercode'),4)
    if __name__ == '__main__':
        pool = Pool(4)
        pool.map(save, enumerate(chunks))

In [18]:
# putting all bands in one, and splitting up by uid
# if __name__ == '__main__':
#     pool = Pool(4)
#     chunks = enumerate([master.loc[:200000], master.loc[200001:320000], master.loc[320001:420000], master.loc[420001:]])
#     pool.map(save, chunks)